In [1]:
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

In [2]:
nlp = spacy.load("en_core_web_trf")

In [3]:
# Add LanguageDetector and assign it a string name
@Language.factory("language_detector")
def create_language_detector(nlp, name):
    return LanguageDetector(language_detection_function=None)

In [4]:
nlp.add_pipe("language_detector")

In [5]:
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

In [6]:
print([(w.text, w.pos_) for w in doc])

[('Apple', 'PROPN'), ('is', 'AUX'), ('looking', 'VERB'), ('at', 'ADP'), ('buying', 'VERB'), ('U.K.', 'PROPN'), ('startup', 'NOUN'), ('for', 'ADP'), ('$', 'SYM'), ('1', 'NUM'), ('billion', 'NUM')]


In [7]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

Apple Apple PROPN NNP nsubj Xxxxx True False
is be AUX VBZ aux xx True True
looking look VERB VBG ROOT xxxx True False
at at ADP IN prep xx True True
buying buy VERB VBG pcomp xxxx True False
U.K. U.K. PROPN NNP compound X.X. False False
startup startup NOUN NN dobj xxxx True False
for for ADP IN prep xxx True True
$ $ SYM $ quantmod $ False False
1 1 NUM CD compound d False False
billion billion NUM CD pobj xxxx True False


In [18]:
for token in doc:
    print(token.text, token.morph.to_dict())

Apple {'Number': 'Sing'}
is {'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin'}
looking {'Aspect': 'Prog', 'Tense': 'Pres', 'VerbForm': 'Part'}
at {}
buying {'Aspect': 'Prog', 'Tense': 'Pres', 'VerbForm': 'Part'}
U.K. {'Number': 'Sing'}
startup {'Number': 'Sing'}
for {}
$ {}
1 {'NumType': 'Card'}
billion {'NumType': 'Card'}


In [8]:
# document level language detection. Think of it like average language of the document!
print(doc._.language)
# sentence level language detection
for sent in doc.sents:
   print(sent, sent._.language)

{'language': 'en', 'score': 0.9999972533899455}
Apple is looking at buying U.K. startup for $1 billion {'language': 'en', 'score': 0.9999944832453606}


In [29]:
for token in doc:
    print(token.text, token.dep_, token.head.text, token.morph.to_dict())

Apple nsubj looking {'Number': 'Sing'}
is aux looking {'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin'}
looking ROOT looking {'Aspect': 'Prog', 'Tense': 'Pres', 'VerbForm': 'Part'}
at prep looking {}
buying pcomp at {'Aspect': 'Prog', 'Tense': 'Pres', 'VerbForm': 'Part'}
U.K. compound startup {'Number': 'Sing'}
startup dobj buying {'Number': 'Sing'}
for prep buying {}
$ quantmod billion {}
1 compound billion {'NumType': 'Card'}
billion pobj for {'NumType': 'Card'}


In [9]:
print(doc)

Apple is looking at buying U.K. startup for $1 billion


In [10]:
nlp_ru = spacy.load("ru_core_news_lg")

In [11]:
nlp_ru.add_pipe("language_detector")

In [25]:
doc_ru = nlp_ru("Юрий хороший программист из города Минска")

In [26]:
print([(w.text, w.pos_) for w in doc_ru])

[('Юрий', 'PROPN'), ('хороший', 'ADJ'), ('программист', 'NOUN'), ('из', 'ADP'), ('города', 'NOUN'), ('Минска', 'PROPN')]


In [27]:
for token in doc_ru:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

Юрий юрий PROPN PROPN nsubj Xxxx True False
хороший хороший ADJ ADJ amod xxxx True False
программист программист NOUN NOUN ROOT xxxx True False
из из ADP ADP case xx True True
города город NOUN NOUN nmod xxxx True False
Минска минск PROPN PROPN appos Xxxxx True False


In [28]:
for token in doc_ru:
    print(token.text, token.dep_, token.head.text, token.morph.to_dict())

Юрий nsubj программист {'Animacy': 'Anim', 'Case': 'Nom', 'Gender': 'Masc', 'Number': 'Sing'}
хороший amod программист {'Case': 'Nom', 'Degree': 'Pos', 'Gender': 'Masc', 'Number': 'Sing'}
программист ROOT программист {'Animacy': 'Anim', 'Case': 'Nom', 'Gender': 'Masc', 'Number': 'Sing'}
из case города {}
города nmod программист {'Animacy': 'Inan', 'Case': 'Gen', 'Gender': 'Masc', 'Number': 'Sing'}
Минска appos города {'Animacy': 'Inan', 'Case': 'Gen', 'Gender': 'Masc', 'Number': 'Sing'}


In [15]:
# document level language detection. Think of it like average language of the document!
print(doc_ru._.language)
# sentence level language detection
for sent in doc_ru.sents:
   print(sent, sent._.language)

{'language': 'ru', 'score': 0.9999953820392045}
Юрий хороший программист {'language': 'ru', 'score': 0.9999971066511354}


In [16]:
doc2 = nlp("Apple is looking at buying U.K. startup for $1 billion. Это предложение не на английском. Третье предложение.")

In [17]:
# document level language detection. Think of it like average language of the document!
print(doc2._.language)
# sentence level language detection
for sent in doc2.sents:
   print(sent, sent._.language)

{'language': 'bg', 'score': 0.8571393245898145}
Apple is looking at buying U.K. startup for $1 billion. Это предложение не на английском. Третье предложение. {'language': 'bg', 'score': 0.9999936468682644}
